In [2]:
import tensorflow as tf 

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import tensorflow as tf
import cv2
import numpy as np

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

In [1]:
# 원래 모델

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)

    outputs = model(image, training=True)
    print(outputs)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./data/fire_smoke/test/JPEGImages/ck0kdhymna0b10721v4wntit8_jpeg.rf.a08e34d04fb672ce6cf8e94e810ec81d.jpg"

if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    efficientdet = EfficientDet()
    efficientdet.load_weights(filepath=Config.save_model_dir + "saved_model")

    image = test_single_picture(picture_dir=test_image_dir, model=efficientdet)

    cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("detect result", image)
    cv2.waitKey(0)

NameError: name 'tf' is not defined

In [ ]:
# tflite 평가 하기

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)
    
    input_data = np.array(image, dtype = np.uint8)
    input_details = model.get_input_details()[0]
    output_details = model.get_output_details()[0]
    model.set_tensor(input_details['index'], input_data)
    model.invoke()
    outputs= model.get_tensor(output_details['index'])
    
    print(outputs)
    # 원래 outputs =  소수점 형태
    # quantizaation model =  숫자 너무큼.. output_tensor shape 해결하기
    
    
    
    outputs = tf.constant(outputs)
    
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./test_pictures/ck0kfhu4n8q7f0701ixmonyig_jpeg.rf.a3cc5282520b3bac90718bdd5528bd76.jpg"

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"
efficientdet_lite = load_model(model_path)

image = test_single_picture(picture_dir=test_image_dir, model=efficientdet_lite)

cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)

In [ ]:
!nvidia-smi